## quadtree & Barnes-Hut

https://www.cs.princeton.edu/courses/archive/fall03/cs126/assignments/barnes-hut.html  
http://www-inf.telecom-sudparis.eu/COURS/CSC5001/new_site/Supports/Projet/NBody/barnes_86.pdf  
https://en.wikipedia.org/wiki/Barnes%E2%80%93Hut_simulation

https://github.com/rdeits/RegionTrees.jl

https://github.com/JuliaArrays/StaticArrays.jl

In [12]:
using BenchmarkTools

In [1]:
mutable struct Cell
    center ::NTuple{2, Float64}
    size ::NTuple{2, Float64}
    nbr_of_points ::Int
    sum_of_points ::NTuple{2, Float64}
    childrens ::Array{Union{Nothing, Cell}, 1}
end;

Cell(center, size) = Cell(
    center,
    size,
    0,
    (0.0, 0.0),
    Array{Union{Nothing, Cell}}(nothing, (4, ))
);

In [2]:
Cell((0.5, 0.5), (1.0, 1.0))

Cell((0.5, 0.5), (1.0, 1.0), 0, (0.0, 0.0), Union{Nothing, Cell}[nothing, nothing, nothing, nothing])


1. If node x does not contain a body, put the new body b here.
2. If node x is an internal node, update the center-of-mass and total mass of x. Recursively insert the body b in the appropriate quadrant.
3. If node x is an external node, say containing a body named c, then there are two bodies b and c in the same region. Subdivide the region further by creating four children. Then, recursively insert both b and c into the appropriate quadrant(s). Since b and c may still end up in the same quadrant, there may be several subdivisions during a single insertion. Finally, update the center-of-mass and total mass of x. 

In [3]:
function which_quadrant(point, cell_center)
    x, y = point
    center_x, center_y = cell_center
    quadrant = 1
    if x > center_x
        quadrant += 1
        end;
    if y < center_y
        quadrant += 2
        end;
    return quadrant
    end;
            
# Numbering:
#   1 | 2
#  -------
#   3 | 4
# cell corner (x, y) is bottom left

In [4]:
@assert which_quadrant((0.3, 0.3), (0.5, 0.5)) == 3

In [5]:
const offset = [(-0.25, 0.25),
                (0.25, 0.25),
                (-0.25, -0.25),
                (0.25, -0.25)]

function create_subcell(cell, quadrant)
    size = cell.size ./ 2 
    center = cell.center .+ (offset[quadrant].*cell.size)
    return Cell(center, size)
    end;

function get_subcell!(cell, point)
    quadrant = which_quadrant(point, cell.center)
    if isnothing(cell.childrens[quadrant])
        cell.childrens[quadrant] = create_subcell(cell, quadrant)
        end;
    return cell.childrens[quadrant]
    end;

function insert!(cell, point)
    # check if node in cell? ... no
    
    if cell.nbr_of_points > 0
    
        if cell.nbr_of_points == 1
            # insert further the old point
            previous_point = cell.sum_of_points
            subcell = get_subcell!(cell, previous_point)
            insert!(subcell, previous_point)
            end;
        
        # continue insertion of the point
        subcell = get_subcell!(cell, point)
        insert!(subcell, point)
        end;
    
    # update cell data
    cell.sum_of_points = cell.sum_of_points .+ point
    cell.nbr_of_points += 1     
    end;

In [6]:
root = Cell((0.5, 0.5), (1.0, 1.0))

Cell((0.5, 0.5), (1.0, 1.0), 0, (0.0, 0.0), Union{Nothing, Cell}[nothing, nothing, nothing, nothing])

In [7]:
function build_the_tree(points)
    root = Cell((0.5, 0.5), (1.0, 1.0))
    for xy in eachcol(points)
        insert!(root, Tuple(xy))
        end;
    return root
    end;

In [8]:
distance(a, b) = sqrt(sum( (a .- b).^2 ));

# evaluate cell, x, y, theta=.5
# Returns list of points (effective) with their mass
function evaluate(cell, point, theta=0.5)
    
    if isnothing(cell) || cell.nbr_of_points == 0
        return Iterators.Stateful([])
    elseif cell.nbr_of_points == 1
        return Iterators.Stateful([(cell.sum_of_points, cell.nbr_of_points), ])
    end
    
    center_of_mass = cell.sum_of_points ./ cell.nbr_of_points
    angle = sum(cell.size)/2/distance(point, center_of_mass)
    
    if angle < theta
        return Iterators.Stateful([(center_of_mass, cell.nbr_of_points), ])
    else
        return Iterators.flatten(evaluate(quad, point, theta) for quad in cell.childrens)
        end;
    end;



In [108]:
points = [Tuple(rand(2)) for k in 1:130];

In [111]:
x_bound = extrema(getindex.(points, 1))
y_bound = extrema(getindex.(points, 2))

(0.004024401841452141, 0.9979409906647885)

In [45]:
using Profile

In [14]:
# Random points
N = 1200
points = rand(2, N);

In [15]:
root = build_the_tree(points);

In [17]:
root.nbr_of_points

1200

In [16]:
@benchmark collect(evaluate(root, xy)) setup=(xy=Tuple(rand(2)))

BenchmarkTools.Trial: 
  memory estimate:  19.38 KiB
  allocs estimate:  377
  --------------
  minimum time:     63.935 μs (0.00% GC)
  median time:      205.427 μs (0.00% GC)
  mean time:        233.155 μs (5.98% GC)
  maximum time:     179.096 ms (2.24% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [18]:
@benchmark evaluate(root, xy) setup=(xy=Tuple(rand(2)))
#   median time:      86.666 ns (0.00% GC)

BenchmarkTools.Trial: 
  memory estimate:  96 bytes
  allocs estimate:  3
  --------------
  minimum time:     80.575 ns (0.00% GC)
  median time:      83.754 ns (0.00% GC)
  mean time:        104.857 ns (16.87% GC)
  maximum time:     62.229 μs (99.81% GC)
  --------------
  samples:          10000
  evals/sample:     971

In [20]:
p = length(collect(evaluate(root, (.2, .2))))

91

In [100]:
@benchmark build_the_tree(points) setup=(points = rand(2, N))
# N 12       median time:      51.217 μs (0.00% GC)
# N 123      median time:      815.989 μs (0.00% GC)
# N 1234     median time:      11.633 ms (0.00% GC)
# N 12345    median time:      163.042 ms (0.00% GC)

LoadError: UndefVarError: @benchmark not defined

In [140]:
root.nbr_of_points

12

In [105]:
# Profile.print()

In [173]:
const EPSILON = 1e-4
function repulsive_energy(x1, y1, x2, y2)
    u = x2 - x1
    v = y2 - y1
    return 1/sqrt(u^2 + v^2 + EPSILON)
    end;

function repulsive_gradient(u, v)
    d = sqrt(u^2 + v^2 + EPSILON)^3
    return (u/d, v/d)
    end;

In [308]:
# brute force
function brute_force_gradient!(G, points)
    fill!(G, 0.f0)
    for (i, (x1, y1)) in enumerate( eachcol(points) )
        for (j, (x2, y2)) in enumerate( eachcol(points[:, i+1:end]) )
            j += i
            u = x2 - x1
            v = y2 - y1
            grad_x, grad_y = repulsive_gradient(u, v)
            G[1, i] += grad_x
            G[2, i] += grad_y
            G[1, j] -= grad_x
            G[2, j] -= grad_y
            end;
        end;
    end;

In [245]:
# Barnes Hut
function barnes_hut_gradient!(G, points, theta=.5)
    fill!(G, 0.f0)
    root = build_the_tree(points)
    for (i, (x1, y1)) in enumerate( eachcol(points) )
        effective_points = evaluate(root, x1, y1, theta)
        for (x2, y2, n) in effective_points
            u = x2 - x1
            v = y2 - y1
            grad_x, grad_y = repulsive_gradient(u, v)
            G[1, i] += n*grad_x
            G[2, i] += n*grad_y
            end;
        end;
    end;

In [247]:
points = [[0.0 1.0]; [0.0 0.0]]
G = zeros(size(points));

In [253]:
points

2×10 Array{Float64,2}:
 0.391266  0.916808  0.432993  0.650874  …  0.949444   0.900131  0.281138
 0.854405  0.426872  0.812079  0.733107     0.0726933  0.322408  0.60046 

In [254]:
brute_force_gradient!(G, points)
G

2×10 Array{Float64,2}:
  231.565  -36.0299  -149.953  -15.7223  …  -17.0361  -14.2239  33.5102
 -199.275  -89.7624   192.798   40.2112      36.5835   74.8512  27.523 

In [371]:
# Random points
N = 100
points = @SMatrix rand(Float32, 2, N)

#G = zeros(Float32, size(points))

2×100 SArray{Tuple{2,100},Float32,2,200} with indices SOneTo(2)×SOneTo(100):
 0.763132  0.172423  0.861806  0.352601   …  0.782554  0.116271  0.777061
 0.466942  0.287075  0.075106  0.0748723     0.978348  0.949976  0.829984

In [372]:
G = @MMatrix zeros(Float32, 2, N)

2×100 MArray{Tuple{2,100},Float32,2,200} with indices SOneTo(2)×SOneTo(100):
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [377]:
c
G = zeros(Float64, 2, N)

2×100 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [379]:
barnes_hut_gradient!(G, points, 0.5)
G

2×100 Array{Float64,2}:
 -491.211    141.603  865.0       783.31  …   41.8299  -425.995  -37.5062 
   98.9384  -418.786   -2.39318  -942.62     476.271     37.811    6.92933

In [378]:
brute_force_gradient!(G, points)
G

2×100 Array{Float64,2}:
 -492.393   138.797  871.037      782.038  …   40.3148  -428.554   -34.6318 
   97.581  -420.924   -0.662454  -944.057     485.641     36.7394    9.01012

In [380]:
@benchmark brute_force_gradient!(G, points)
#   median time:      193.477 μs (0.00% GC)  with Marray
#   median time:      155.845 μs (0.00% GC)  with Array Float64

BenchmarkTools.Trial: 
  memory estimate:  92.97 KiB
  allocs estimate:  200
  --------------
  minimum time:     152.928 μs (0.00% GC)
  median time:      155.845 μs (0.00% GC)
  mean time:        169.034 μs (4.57% GC)
  maximum time:     4.428 ms (95.57% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [374]:
@benchmark barnes_hut_gradient!(G, points, 0.5)

BenchmarkTools.Trial: 
  memory estimate:  6.59 MiB
  allocs estimate:  257289
  --------------
  minimum time:     27.994 ms (0.00% GC)
  median time:      33.146 ms (0.00% GC)
  mean time:        36.588 ms (6.29% GC)
  maximum time:     64.889 ms (29.63% GC)
  --------------
  samples:          137
  evals/sample:     1

In [381]:
@benchmark barnes_hut_gradient2!(G, points, 0.5)

BenchmarkTools.Trial: 
  memory estimate:  1.18 MiB
  allocs estimate:  68142
  --------------
  minimum time:     2.538 ms (0.00% GC)
  median time:      2.615 ms (0.00% GC)
  mean time:        2.913 ms (7.99% GC)
  maximum time:     12.434 ms (76.43% GC)
  --------------
  samples:          1712
  evals/sample:     1

In [351]:
# evaluate cell, x, y, theta=.5
# Returns list of points (effective) with their mass
function evaluate2(cell, x, y, fun, theta=.5)
    
    if cell.nbr_of_points != 0
        
        center_of_mass_x = cell.sum_of_points_x / cell.nbr_of_points
        center_of_mass_y = cell.sum_of_points_y / cell.nbr_of_points

        distance = get_distance(x, y, center_of_mass_x, center_of_mass_y)
        angle = (cell.w + cell.h)/2/distance

        if angle < theta || cell.nbr_of_points == 1
            fun(center_of_mass_x, center_of_mass_y, cell.nbr_of_points)
        else
            for quad in cell.childrens
                evaluate2(quad, x, y, fun, theta)
                end;
        
            end;
                    
    
        end;
    end;

# brute force
function barnes_hut_gradient2!(G, points, theta=.5)
    fill!(G, 0.f0)
    root = build_the_tree(points)
    for (i, (x1, y1)) in enumerate( eachcol(points) )
        
        function fun(x2, y2, n)
            u = x2 - x1
            v = y2 - y1
            grad_x, grad_y = repulsive_gradient(u, v)
            G[1, i] += n*grad_x
            G[2, i] += n*grad_y
            end;
        evaluate2(root, x1, y1, fun, theta)
        end;
    end;


# Barnes Hut 3
        


function accumulate_gradient!(G, i, x1, y1, x2, y2, n)
    u = x2 - x1
    v = y2 - y1
    grad_x, grad_y = repulsive_gradient(u, v)
    G[1, i] += n*grad_x
    G[2, i] += n*grad_y
    end;

function barnes_hut_gradient3!(G, points, theta=.5)
    fill!(G, 0.f0)
    root = build_the_tree(points)
    for (i, (x1, y1)) in enumerate( eachcol(points) )
        evaluate3!(G, root, i, x1, y1, theta)
        end;
    end;

function evaluate3!(G, cell, i, x1, y1, theta=.5)
    
    if cell.nbr_of_points != 0
        
        center_of_mass_x = cell.sum_of_points_x / cell.nbr_of_points
        center_of_mass_y = cell.sum_of_points_y / cell.nbr_of_points

        distance = get_distance(x1, y1, center_of_mass_x, center_of_mass_y)
        angle = (cell.w + cell.h)/2/distance

        if angle < theta || cell.nbr_of_points == 1
            #accumulate_gradient!(G, i, x1, y1, center_of_mass_x, center_of_mass_y, cell.nbr_of_points)
            n = cell.nbr_of_points
            u = center_of_mass_x - x1
            v = center_of_mass_y - y1
            grad_x, grad_y = repulsive_gradient(u, v)
            G[1, i] += n*grad_x
            G[2, i] += n*grad_y
        else
            for quad in cell.childrens 
                evaluate3!(G, quad, i, x1, y1, theta)
                end;
        
            end;
                    
    
        end;
    end;

In [352]:
barnes_hut_gradient2!(G, points, .5)
G

2×1000 Array{Float32,2}:
 2722.28  5075.01  -2374.55  -2576.62  …  -3940.12  -6806.39   -961.307
 3712.06   315.58  -3314.14  -5254.99     -1286.82  -1155.95  -4612.31 

In [344]:
brute_force_gradient!(G, points)
G

2×1000 Array{Float32,2}:
 2713.85  5158.2    -2379.62  -2594.59  …  -3959.17  -6870.55   -932.363
 3749.46   365.158  -3371.21  -5307.76     -1231.82  -1145.16  -4678.95 

In [345]:
barnes_hut_gradient3!(G, points)
G

2×1000 Array{Float32,2}:
 2722.28  5075.01  -2374.55  -2576.62  …  -3940.12  -6806.39   -961.307
 3712.06   315.58  -3314.14  -5254.99     -1286.82  -1155.95  -4612.31 

In [353]:
@benchmark barnes_hut_gradient2!(G, points, 1.)
#   median time:      29.461 ms (0.00% GC)
#  median time:      31.904 ms (0.00% GC)  with checking children

BenchmarkTools.Trial: 
  memory estimate:  9.49 MiB
  allocs estimate:  557419
  --------------
  minimum time:     29.388 ms (0.00% GC)
  median time:      29.773 ms (0.00% GC)
  mean time:        31.945 ms (5.63% GC)
  maximum time:     42.292 ms (20.49% GC)
  --------------
  samples:          157
  evals/sample:     1

In [346]:
@benchmark barnes_hut_gradient3!(G, points, 1.)

BenchmarkTools.Trial: 
  memory estimate:  16.88 MiB
  allocs estimate:  1010453
  --------------
  minimum time:     48.850 ms (0.00% GC)
  median time:      49.490 ms (0.00% GC)
  mean time:        52.836 ms (5.71% GC)
  maximum time:     62.961 ms (12.65% GC)
  --------------
  samples:          95
  evals/sample:     1

In [341]:
@benchmark brute_force_gradient!(G, points)

BenchmarkTools.Trial: 
  memory estimate:  4.00 MiB
  allocs estimate:  2000
  --------------
  minimum time:     16.129 ms (0.00% GC)
  median time:      16.258 ms (0.00% GC)
  mean time:        16.633 ms (2.03% GC)
  maximum time:     20.311 ms (18.36% GC)
  --------------
  samples:          301
  evals/sample:     1

In [319]:
@benchmark build_the_tree(points)

BenchmarkTools.Trial: 
  memory estimate:  1.22 MiB
  allocs estimate:  66590
  --------------
  minimum time:     9.173 ms (0.00% GC)
  median time:      9.323 ms (0.00% GC)
  mean time:        9.669 ms (2.28% GC)
  maximum time:     19.198 ms (47.96% GC)
  --------------
  samples:          517
  evals/sample:     1

In [388]:
# Random points
N = 10000
points = rand(Float32, 2, N)

2×10000 Array{Float32,2}:
 0.692833  0.812582   0.483143  0.629875  …  0.940152  0.875751  0.527805
 0.487994  0.0752831  0.858594  0.487042     0.100747  0.552562  0.591484

In [403]:
@benchmark xy = points[[1, 2], 633]
#   median time:      134.383 ns (0.00% GC)
#   median time:      294.783 ns (0.00% GC)
#   median time:      391.080 ns (0.00% GC)  individual access
#   median time:      678.156 ns (0.00% GC)    points = rand(Float32, 2, N)
#   median time:      1.102 μs (0.00% GC)  points = rand(Float32, N, 2)

BenchmarkTools.Trial: 
  memory estimate:  224 bytes
  allocs estimate:  3
  --------------
  minimum time:     126.405 ns (0.00% GC)
  median time:      134.332 ns (0.00% GC)
  mean time:        183.348 ns (24.03% GC)
  maximum time:     11.125 μs (98.59% GC)
  --------------
  samples:          10000
  evals/sample:     911

In [399]:
xy = points[[1, 2], 6363]

2-element Array{Float32,1}:
 0.013183832
 0.4978845  

In [397]:
xy

2-element Array{Float32,1}:
 0.013183832
 0.4978845  